In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [174]:
df = pd.read_csv("data.csv", sep = ";")

In [175]:
df["percent.1"] = df["percent.1"].apply(lambda x: round(float(x.replace(",", "."))*2/10)/2*10)

In [176]:
df['prec'] = df['prec'].apply({'snow':2, 'rain':1, "storm":3,np.nan : 0}.get) 

In [177]:
df = df.drop(["Unnamed: 0", "absenteeism", "percent","percentNormalized"], axis = 1)

In [178]:
df

,date,year,mounth,day,weekday,temp,pres,prec,wind,percent.1
0,20120114,2012,1,14,6,-21,756,0,1.0,20.0
1,20120116,2012,1,16,1,-19,758,0,2.0,15.0
2,20120117,2012,1,17,2,-27,770,0,1.0,15.0
3,20120118,2012,1,18,3,-28,776,0,2.0,30.0
4,20120119,2012,1,19,4,-27,778,0,2.0,20.0
...,...,...,...,...,...,...,...,...,...,...
1689,20181225,2018,12,25,2,-23,770,0,1.0,10.0
1690,20181226,2018,12,26,3,-22,770,0,NaN,5.0
1691,20181227,2018,12,27,4,-23,770,0,NaN,5.0
1692,20181228,2018,12,28,5,-25,766,0,NaN,0.0


In [179]:
X = df.drop(["date", "percent.1"], axis =1)
X["prec"].value_counts()

0    1457
2     174
1      57
3       6
Name: prec, dtype: int64

In [180]:
X = pd.isnull(X)

In [181]:
y = df["percent.1"]

In [182]:
y

0       20.0
1       15.0
2       15.0
3       30.0
4       20.0
        ... 
1689    10.0
1690     5.0
1691     5.0
1692     0.0
1693     0.0
Name: percent.1, Length: 1694, dtype: float64

In [183]:
y.value_counts()

15.0    479
10.0    354
20.0    276
5.0     206
0.0     158
25.0    114
30.0     58
35.0     24
40.0     12
45.0      9
50.0      4
Name: percent.1, dtype: int64

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [185]:
kfold = StratifiedKFold(n_splits=3) # задаем объект для разделения данных на фолды при кросс-валидации

model = DecisionTreeClassifier()

params = {
          'max_depth': range(1,50,2)
         }  # перебираемые параметры модели

grid = GridSearchCV(model, params, cv=kfold, scoring='accuracy', n_jobs=-1)

In [186]:
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
       

In [187]:
mod = grid.best_estimator_
mod

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=1, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [188]:
mod = grid.best_estimator_
y_test_pred = mod.predict(X_test)
accuracy_score(y_test, y_test_pred)

0.2829076620825147